In [ ]:
!pip install -e /Users/esummer/workspace/python/my-python-utils-main

In [ ]:
!pip install slugify

!pip install pandas
!pip install selenium
!pip install openpyxl

!pip install requests pandas xmltodict
!pip install etree
!pip install lxml

!pip install --upgrade pip


In [ ]:
import json
import os

from slugify import slugify
import csv

import logging
import time
import random
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

import pandas as pd
import time
import requests
import xml.etree.ElementTree as ET

from zoneinfo import ZoneInfo

import re

In [ ]:
df_product_product = pd.read_excel('./import/품목 세부사항 (product.product) (3).xlsx')
df_apx_product_product = pd.read_excel('./import/Product Variant (product.product) (2).xlsx')
# df_income = pd.read_excel('덕현테니스클럽 2025 (1).xlsx', header=2, sheet_name='2025년 수입 세부내역')

In [ ]:
df_product_product.head(100000)

In [ ]:
df = df_product_product.copy()
df = df_apx_product_product.copy()
# 'a' 열을 기준으로 그룹화하여 각 그룹의 데이터 개수를 계산합니다.
# size()는 각 그룹의 행 개수를 반환하며, count()는 결측치(NaN)를 제외한 개수를 반환합니다.
# result = df.groupby('품목 코드').size().reset_index(name='count')
result = df.groupby('Internal Reference').size().reset_index(name='count')

# 결과 확인
result_filtered = result[result['count'] >= 2]
print(result_filtered)

# df_a와 df_b를 'aa' 컬럼 기준으로 병합
result = pd.merge(result_filtered, df_apx_product_product[['Internal Reference', 'Name', 'Attribute Values']], on='Internal Reference', how='inner')

result.to_excel('품목 코드_groupby_result_sh.xlsx', index=False)

#
# # 구분2 값이 시설비 적립 인것 제거
# df_payout_sum = df_payout_sum[df_payout_sum['구분2'] != '시설비 적립']
# df_payout_sum['_x0008_날짜'] = pd.to_datetime(df_payout_sum['_x0008_날짜'])
# df_payout_sum['연월'] = df_payout_sum['_x0008_날짜'].dt.strftime('%Y년 %m월')
#
# df_income_sum = df_income_sum[df_income_sum['구분'] != '전년도 이월']
# df_income_sum['_x0008_날짜'] = pd.to_datetime(df_income_sum['_x0008_날짜'])
# df_income_sum['연월'] = df_income_sum['_x0008_날짜'].dt.strftime('%Y년 %m월')
#
# df_payout_sum.info()

# 여기까지 사용

## 국가유산청 Open API

In [ ]:
import pandas as pd
import time
import requests
import xml.etree.ElementTree as ET

df = pd.DataFrame()
error_urls = []
for i in range(1, int(17737 / 50) +1 ):
    url = f'http://www.khs.go.kr/cha/SearchKindOpenapiList.do?pageUnit=50&pageIndex={i}'

    try:
        # 2. 데이터 가져오기 (requests 이용)
        response = requests.get(url)
        response.raise_for_status()  # 200 OK가 아니면 에러 발생

        # 3. XML 데이터를 데이터프레임으로 변환
        # 국가유산청 API 결과에서 각 항목은 <item> 태그 안에 들어있으므로 xpath를 지정합니다.
        df_a = pd.read_xml(response.content, xpath=".//item")
        df = pd.concat([df, df_a], ignore_index=True)

        df_a.to_excel(f'{i:04d}_SearchKindOpenapiList.xlsx')
        # 4. 결과 확인
        print(f"데이터 크기: {df.shape}")
        # print(df.head())
        time.sleep(1)
    except Exception as e:
        print(f"오류 발생: {e}")
        error_urls.append(url)
    pass


In [ ]:
df.to_excel("국가유산.xlsx", index=False)

## 국가유산 검색 상세

In [ ]:
df_base = pd.read_excel("국가유산.xlsx")

In [ ]:
# import ET
from pathlib import Path
from datetime import datetime
import pytz

# 한국 타임존 설정
korea_timezone = pytz.timezone('Asia/Seoul')

step_count = 3000
step = 1 +1+1+1+1+1

start = ((step -1) * step_count)

print(start, start + step_count)
for index, row in df_base.iterrows() :

    if index < start + step_count and index >= start:
        ccbaKdcd = row['ccbaKdcd']
        ccbaAsno = row['ccbaAsno']
        ccbaCtcd = row['ccbaCtcd']
        file_name = row['ccbaMnm1']

        ccbaAsno = f'0000000{ccbaAsno}'
        ccbaAsno = ccbaAsno[-13:]

        surl = f"https://www.khs.go.kr/cha/SearchKindOpenapiDt.do?ccbaKdcd={ccbaKdcd}&ccbaAsno={ccbaAsno}&ccbaCtcd={ccbaCtcd}"

        if (index % 50 == 0):
            # 현재 시간 가져오기
            now_korea = datetime.now(korea_timezone)
            # 출력 형식 지정
            formatted_time = now_korea.strftime('%Y-%m-%d %H:%M:%S')
            print("", formatted_time, index+1, surl)

        try:
            file_path = Path(f'./detail/{index:05d}_상세_{file_name}.xlsx')

            # 1. 파일 또는 디렉토리 존재 여부 전체
            if file_path.exists():
                continue

            print("*", end="")

            # 2. 데이터 가져오기 (requests 이용)
            response = requests.get(surl)
            response.raise_for_status()

            df_a = pd.read_xml(response.content, xpath=".//item")
            df = pd.concat([df, df_a], ignore_index=True)

            df_a.to_excel(file_path)

        except Exception as ex:
            print(ex)
            print(index, surl)

## 국가유산 목록에서 이미지파일 조회

In [ ]:
df = pd.read_excel("국가유산.xlsx")
# df.head()

In [ ]:
# import ET
from pathlib import Path
step = 1000
step = 10

start = 1000 +500 +500 +500 +500 +500 +1000 +1000 +step+step
start = 11500+step
start = 6500
start = 17500
start = 0

for index, row in df.iterrows() :

    if index < start + step and index >= start:
        ccbaKdcd = row['ccbaKdcd']
        ccbaAsno = row['ccbaAsno']
        ccbaCtcd = row['ccbaCtcd']
        file_name = row['ccbaMnm1']

        ccbaAsno = f'0000000{ccbaAsno}'
        ccbaAsno = ccbaAsno[-13:]

        surl = f'https://www.khs.go.kr/cha/SearchImageOpenapi.do?ccbaKdcd={ccbaKdcd}&ccbaAsno={ccbaAsno}&ccbaCtcd={ccbaCtcd}'
        if(index % 20 == 0):
            # print(index, surl)
            pass
        try:
            file_path = Path(f'./output/{index:05d}_image_{file_name}.xlsx')

            # 1. 파일 또는 디렉토리 존재 여부 전체
            if file_path.exists():
                continue


            # 2. 데이터 가져오기 (requests 이용)
            response = requests.get(surl)
            response.raise_for_status()

            items = response.content

            root = ET.fromstring(response.content)

            result_list = []
            current_item = {}

            # 2. 모든 하위 요소를 순회하며 sn을 기준으로 그룹화
            for child in root:

                # print(child.tag, child.text)

                if child.tag == 'item':
                    for child2 in child:
                        # print(child2.tag, child2.text)
                        # 새로운 <sn>을 만나면 이전까지 쌓인 current_item을 리스트에 추가
                        if child2.tag == 'sn' and current_item:
                            result_list.append(current_item)
                            current_item = {}  # 초기화

                        # 데이터 추가 (태그 이름을 키로, 텍스트를 값으로)
                        current_item[child2.tag] = child2.text

                    # 3. 마지막에 남은 항목 추가
                    if current_item:
                        result_list.append(current_item)

            if(index % 20 == 0):
                print(result_list)

            df2 = pd.DataFrame(result_list)
            df2.to_excel(f'./output/{index:05d}_image_{file_name}.xlsx')
        except Exception as ex:
            print(ex)
            print(index, surl)


In [ ]:
df = pd.read_excel('이미지_국가유산___.xlsx')

In [ ]:
words = "석탑,입상,좌상,칠성도,석등,극락전,극락보전,전탑,무량수전,불국사,해인사,송광사,통도사,백담사,동화사,벽화,대웅전,승탑,법주사,여래,국사탑,사탑,괘불탱,탱화,동종,반가상,미륵불,보살,나한상,괘불".split(",")
words = "교회,성당,예배".split(",")
words = "나무,수림".split(",")
words = "박물관".split(",")

files = []

for index, row in df.iterrows() :
    ccbaKdcd = row['ccbaKdcd']
    ccbaAsno = row['ccbaAsno']
    ccbaCtcd = row['ccbaCtcd']

    # df.at[index, 'images'] = 0

    file_name = row['ccbaMnm1']
    if any(word in file_name for word in words):
        readfile = f'{index:05d}_image_{file_name}.xlsx'

        directory = file_name.replace(":","^").replace("/","&").replace(" ","_").replace("\t","_")

        images = []

        try:
            df2 = pd.read_excel(f"./output/{readfile}")
            for ii, rr in df2.iterrows():
                # if ii > 9:
                #     break
                data = {}
                data['url'] = rr['imageUrl']

                if(data['url'].find('no_image') > -1):
                    continue

                ext = data['url'].split('.')[-1]
                sn = rr['sn']
                name = f"{rr['ccimDesc']}".replace(":","^").replace("/","&").replace(" ","_").replace("\t","_")
                data['name'] = f'{directory}@@{name}##{sn}.{ext}'
                images.append(data)
        except Exception as ex:
            print(ex)
        finally:
            if len(images) > 0:
                df.at[index, 'read_file'] = readfile
                df.at[index, 'images'] = len(images)
                df.at[index, f'images_type'] = "|".join(words[:3])
                files.append(readfile)

    # if(index > 10000):
    #     break

df3 = pd.DataFrame(files, columns=['Name'])
df3.to_excel(f'{"".join(words)}___.xlsx')
df.to_excel(f'이미지_국가유산___.xlsx')
print("파일갯수",len(files),"\n", "Latest\n","\n".join(files[-10:]))

In [ ]:
# 1. 필터링할 단어 리스트 정의
words = "석탑,입상,좌상,칠성도,석등,극락전,극락보전,전탑,무량수전,불국사,해인사,송광사,통도사,백담사,동화사,벽화,대웅전,승탑,법주사,여래,국사탑,사탑,괘불탱,탱화,동종,반가상,미륵불,보살,나한상,괘불".split(",")

# 2. 정규표현식 패턴 생성 (단어들 중 하나라도 포함된 경우)
pattern = "|".join(words)

# 3. ccbaMnm1 컬럼에서 해당 단어가 포함된 데이터 필터링
# na=False를 설정하여 NaN 값이 있는 경우 에러 방지
filtered_df = df[df['ccbaMnm1'].str.contains(pattern, na=False)].copy()

result_df = filtered_df.groupby(['ccbaCtcdNm', 'ccsiName', 'ccbaAdmin'])['ccbaMnm1'].agg(
    count='count',
    all_name=lambda x: "|".join(x.astype(str))
).reset_index()
result_df.to_excel('지역_국가유산___.xlsx')

#
# df_area = df.groupby(['ccbaCtcdNm', 'ccsiName', 'ccbaAdmin']).agg(
#     count_ccbaMnm1=('ccbaMnm1', 'count'),  # 개수 검사
#     all_name=('ccbaMnm1', lambda x: "|".join(x.astype(str)))  # "|" 구분자로 합치기
# ).reset_index()
# df_area.to_excel('지역_국가유산___.xlsx')

In [ ]:
df_income_sum['집계용구분'] = df_income_sum['구분'] + '_'+ df_income_sum['구분2']

df_income_sum_grouped = df_income_sum.groupby('구분').sum(numeric_only=True).reset_index()
df_income_sum_grouped[['구분', '수입']].to_excel('수입_구분_집계.xlsx', index=False)

df_income_sum_grouped = df_income_sum.groupby('집계용구분').sum(numeric_only=True).reset_index()
df_income_sum_grouped[['집계용구분', '수입']].to_excel('수입_집계용구분_집계.xlsx', index=False)

df_income_sum_grouped = df_income_sum.groupby('연월').sum(numeric_only=True).reset_index()
df_income_sum_grouped[['연월', '수입']].to_excel('수입_연월_집계.xlsx', index=False)

In [ ]:
df_payout_sum['집계용구분'] = df_payout_sum['구분'] + '_'+ df_payout_sum['구분2']

df_payout_sum_grouped = df_payout_sum.groupby('구분').sum(numeric_only=True).reset_index()
df_payout_sum_grouped[['구분', '지출']].to_excel('구분_집계.xlsx', index=False)

df_payout_sum_grouped = df_payout_sum.groupby('집계용구분').sum(numeric_only=True).reset_index()
df_payout_sum_grouped[['집계용구분', '지출']].to_excel('집계용구분_집계.xlsx', index=False)


df_payout_sum_grouped = df_payout_sum.groupby('연월').sum(numeric_only=True).reset_index()
df_payout_sum_grouped[['연월', '지출']].to_excel('연월_집계.xlsx', index=False)


In [ ]:
# df = sh_products.copy()
dff = sh_products[sh_products['이름'].notna() & (sh_products['이름'] != '')]

# dff.info()

# Title	Internal Reference	Seller SKU		Fulfillment

dff.rename(columns={
    '이름': 'Title',
    '품목 코드': 'Internal Reference'
}, inplace=True)

dff = dff[['Title', 'Internal Reference']]
dff.info()

dff['Marketplace'] = 'Amazon.com'
dff['Fulfillment'] = 'FBA'
dff['Seller SKU'] = dff['Internal Reference']

dff[['Title','Internal Reference','Seller SKU','Marketplace','Fulfillment']].to_excel(f'../품목_세부선택_(product.product)_{len(dff)}건.xlsx', index=False)

values = ['PAS02XLB', 'PAS04LB', 'PX2CL116', 'PX2DGKB', 'PX2H3ES11', 'PX2H3ES21',
           'PX2SGB', 'PX3LCL101', 'PX3LCL102', 'PX3RCL101', 'PX3RCL102',
           'PXL42QCS', 'PXV321D']

# a 컬럼 값이 리스트에 포함된 행만 추출
filtered_df = dff[dff['Internal Reference'].isin(values)]

filtered_df[['Title','Internal Reference','Seller SKU','Marketplace','Fulfillment']].to_excel(f'../품목_세부선택_(product.product)_{len(filtered_df)}건.xlsx', index=False)


In [ ]:
products = pd.read_csv('../product_template_csv_unit_price (4).csv')


In [ ]:
products.info()
df = products.copy()

dff = df[df['Title'].notna() & (df['Title'] != '')]
dff.to_csv('Title_Exists.csv', index=False)

In [ ]:
dff.info()

In [ ]:
import pandas as pd
import glob
import os

# CSV 파일들이 들어있는 디렉토리 경로
path = '../csv'  # 예: './csv' or 'C:/Users/you/Documents/csv_files'

# 디렉토리 내의 모든 CSV 파일 경로 가져오기
csv_files = glob.glob(os.path.join(path, '*.csv'))

# 파일 읽어서 하나의 리스트로 저장
dfs = pd.DataFrame()

for file in csv_files:
    df = pd.read_csv(file)
    print(file, df.shape)
    # dfs += df
    dfs = pd.concat([dfs, df], ignore_index=True)


# 모든 DataFrame을 하나로 합치기
# merged_df = pd.concat(dfs, ignore_index=True)

# 결과 확인
print(f"총 {len(csv_files)}개의 CSV 파일이 병합되었습니다.")
print(dfs.head())

In [ ]:
dfs.info()

dfs.groupby('Ship to').size().reset_index(name='count').to_excel("../amazon-ship-to.xlsx", index=False)

count_df = dfs['Ship to'].value_counts().reset_index()

# 2️⃣ 컬럼 이름 정리
count_df.columns = ['Ship to', 'count']

# 3️⃣ a 기준으로 오름차순 정렬
count_df = count_df.sort_values(by='Ship to').reset_index(drop=True)

# 4️⃣ 결과 확인
print(count_df.head())

# 5️⃣ CSV로 저장
count_df.to_excel("../amazon-ship-to-2.xlsx", index=False)



In [ ]:
# df_catetorys = pd.read_excel('~esummer/Documents/ko_KR.xlsx',header=1)
df_products = pd.read_excel('품목 세부선택 (product.product) (1).xlsx',header=1)

In [ ]:
df_catetorys.info()

filtered_df = df_catetorys[df_catetorys['value'].notna()]

df = filtered_df[['value', 'src']]

df['value'] = df['value'].astype(str)

df = df.drop_duplicates()

df_sorted = df.sort_values(by='value')

df_sorted.to_excel('odoo_translate.xlsx')

In [ ]:
df_products = pd.read_excel('품목 세부선택 (product.product) (2).xlsx',header=0)

In [ ]:
df_products.info()

In [ ]:
filtered_df = df_catetorys[df_catetorys['value'].notna()]

df = filtered_df[['value', 'src']]

df['value'] = df['value'].astype(str)

df = df.drop_duplicates()

df_sorted = df.sort_values(by='value')

df_sorted.to_excel('odoo_translate.xlsx')

In [ ]:
my_items = '''MMKGSVE04BK001
MMKGSVE05BK001
MMKGSVE06BK001
MMKGSVE08CO001
MMKGSVE09BK001
MMLGSTA02BK001
MMLGSTA03BK001
MMRGSAK01BK001
MMRGSAK01CO001
MMRGSAK02BK001
MMRGSAK02CO001
MMRGSTA05BK001
MMRGSTA06SV001
MMRGSTA07PZ001
MMTGSTA01CO001
MMTGSTA02CO001
MMTGSTA03BK001
MMTGSTA04BK001
MMTGSTA04CO001
MMTGSTA05CO001
MMZGSFI01BK001
MMZGSFI02BK001
MMZGSFI03CO001
MWBGSEA01BK001
MWBGSEA03Q6001
MWBGSEA03QA001
MWBGSEA04CO001
MWBGSEA05BK001
MWBGSEA06BK001
MWBGSEA07BK001
MWCGSAK01BK001
MWDGSDU01BK001
MWDGSDU01Q6001
MWDGSDU01QA001
MWDGSDU02CO001
MWDGSDU03CO001
MWEGSXT01CO001
MWHGSTA03BK001
MWHGSTA03Q6001
MWHGSTA03QA001
MWHGSTA04BK001
MWHGSTA04PZ001
MWKGSAK01BK001
MWKGSTA01Q6001
MWKGSTA01QA001
MWPGSAC01L8001
MWPGSLR02BK001
MWPGSLR02CO001
MWPGSLR02PZ001
MWPGSLR03BK001
MWPGSLR03CO001
MWPGSLR03PZ001
MWPGSLR04BK001
MWPGSLR04CO001
MWPGSLR05BK001
MWPGSLR05CO001
MWPGSMT04BK001
MWPGSMT04QA001
MWPGSMT05BK001
MWPGSMT06CO001
MWPGSMT07CO001
MWPGSMT10BK001
MWPGSMT12L8001
MWPGSTA01PZ001
MWRGSTA01BK001
MWRGSTA01OQ001
MWRGSTA02CO001
MWRGSTA02PZ001
MWRGSTA03BK001
MWRGSXT01BK001
MWRGSXT01CO001
MWSGSAK01BK001
MWSGSTA01BK001
MWSGSTA01OQ001
MWSGSTA02CO001
MWSGSTA04BW001
MWSGSTA05BW001
MWSGSTA05BW001
MWTGSBO02BK001
MWTGSMA01BK001
MWTGSMA02BK001
MWTGSMA02QA001
MWTGSTA02BK001
MWTGSTA03PZ001
MWZGSAK01BK001
MXBGSCJ01BK001
MXBGSCJ02CO001
MXBGSTA02BK080
MXBGSTA03BK080
MXBGSTA04BK001
MXBGSTA05BK001
MXZFAAK01BK001
MXZGSAK02BK001
MXZGSKS01BK001
MYZDSTA09PZ001
MYZFSTA01Q6001
MYZGSTA01CO001
MYZGSTA02CO001
MYZGSTA03BK001
MYZGSTA04L8001
MYZGSTA05CO001
MYZGSTA06BK001
MYZGSTA06CO001
MYZGSTA07QA001
MYZGSTA08PZ001'''.split('\n')



import random


# 임의로 10개 선택
selected_items = random.sample(my_items, 10)

print(selected_items)

df = pd.DataFrame(selected_items, columns=['SKU'])

## 기준정보 테이블 읽기

In [ ]:
df_products = pd.read_excel('../excel-input/00.기준정보 테이블__Products.xlsx', header=0)
df_regions = pd.read_excel('../excel-input/00.기준정보 테이블__Regions.xlsx', header=0)
df_vendor = pd.read_excel('../excel-input/00.기준정보 테이블__Vendors.xlsx', header=0)
df_catetorys = pd.read_excel('../excel-input/00.기준정보 테이블__Catetorys.xlsx', header=0)

## Production 마스터 데이터 읽어 오기

In [ ]:
df_production_origin = pd.read_excel('../excel-input/00.3.Production 마스터 (생산본부)__Production Sample.xlsx', header=0)
df_production_origin.info()

In [ ]:
filtered_notna = df_production_origin[df_production_origin['PO  Q\'ty'].notna()]

# filtered_notna[:10].to_json('../excel-output/df_production_sample_template#1.json', orient='records')
filtered_notna[:10][['item',"PO  Q'ty", '(Contractual) Ex Fty Month','Category','Line Name', 'Order Type','Program','(GMD) Contractual Ex fty','Vendor', 'Channel']].to_json('../excel-output/df_production_sample_template#1_filter.json', orient='records')


## 주문 데이터의 각 컬럼별 info()

In [ ]:
from io import StringIO

df_production_sample = df_production_origin.copy()

df_production_sample.columns = df_production_sample.columns.str.replace(' ', '__')


buf = StringIO()
df_production_sample.info(buf=buf)
info_str = buf.getvalue()

# 2. info 문자열을 줄 단위로 나눔
lines = info_str.strip().split("\n")

# 3. 컬럼 정보 라인만 추출 (인덱스, 컬럼명, Non-Null Count, Dtype)
data_lines = [line.strip() for line in lines if "non-null" in line]

# 4. 공백 기준 분리 후 DataFrame 생성
info_data = []
for line in data_lines:
    parts = line.split()
    col_name = parts[1]
    non_null = parts[2]
    dtype = parts[-1]
    info_data.append([col_name, non_null, dtype])

info_df = pd.DataFrame(info_data, columns=["Column", "Non-Null Count", "Dtype"])

# 5. 엑셀로 저장
info_df.to_excel("../excel-output/df_production_sample_info.xlsx", index=False)
df_production_sample.to_excel("../excel-output/df_production_sample_data.xlsx", index=False)

## 코드성 컬럼값 추출

In [ ]:
# a 컬럼의 값 종류별 카운트
columns = ['Order__Type', 'Program','Category',	'Vendor',	'Channel',	'Line__Name', 'PO__No__(SAP)', 'PO__No']
for column in columns:
    count_df = df_production_sample[column].value_counts().reset_index()

    # 컬럼명 변경
    count_df.columns = [f"{column} 값", "count"]
    count_df.to_excel(f"../excel-output/df_production_컬럼값_{column}_data-{len(count_df)}-건.xlsx", index=False)

In [ ]:
df_production_sample[:10].to_json('../excel-output/df_production_sample_template_#2.json', orient='records')

In [ ]:
df_production_sample_2 = pd.read_excel('../excel-input/★1. Q12 26(SS26) Selling_initial_out_0630_V2_Sample.xlsx')
df_production_sample_2.info()

filtered_notna = df_production_sample_2[df_production_sample_2['WW ORDER QTY'].notna()]

# filtered_notna[:10].to_json('../excel-output/df_production_sample_2_all.json', orient='records')
filtered_notna[:10][['ITEM#','WW ORDER QTY','MOQ','EX-FTY DATE','CATEGORY','LINE NAME','GMD MASTER ORDER','LAUNCHING MONTH','DEPARTMENT','OTB Cate']].to_json('../excel-output/df_production_sample_template#2_filter.json', orient='records')

## 코드성 컬럼의 숫자 집계

In [ ]:
unique_values = df_production_sample[['Order__Type', 'Program']].drop_duplicates()
unique_values.sort_values(by=['Order__Type', 'Program'], ascending=[True,True]).to_excel(f"../excel-output/df_production_컬럼값_[Order__Type & Program]_data-{len(unique_values)}-건.xlsx", index=False)

unique_values = df_production_sample[['Order__Type', 'Category']].drop_duplicates()
unique_values.sort_values(by=['Order__Type', 'Category'], ascending=[True,True]).to_excel(f"../excel-output/df_production_컬럼값_[Order__Type & Category]_data-{len(unique_values)}-건.xlsx", index=False)

unique_values = df_production_sample[['Category', 'Order__Type']].drop_duplicates()
unique_values.sort_values(by=['Category', 'Order__Type'], ascending=[True,True]).to_excel(f"../excel-output/df_production_컬럼값_[Category & Order__Type]_data-{len(unique_values)}-건.xlsx", index=False)

unique_values = df_production_sample[['Program', 'Order__Type']].drop_duplicates()
unique_values.sort_values(by=['Program', 'Order__Type'], ascending=[True,True]).to_excel(f"../excel-output/df_production_컬럼값_[Program & Order__Type]_data-{len(unique_values)}-건.xlsx", index=False)


## 코드성 컬럼의 그룹바이 집계

In [ ]:
counts_groupby = df_production_sample.groupby(['Category', 'Order__Type']).size().reset_index(name='건수')
counts_groupby.to_excel(f"../excel-output/df_production_컬럼값_[Category & Order__Type]_groupby-{len(counts_groupby)}-건.xlsx", index=True)

In [ ]:
counts_groupby = df_production_sample.groupby(['Line__Name', 'item']).size().reset_index(name='건수')
counts_groupby.to_excel(f"../excel-output/df_production_컬럼값_[Line__Name & item]_groupby-{len(counts_groupby)}-건.xlsx", index=True)

counts_groupby = df_production_sample.groupby(['item', 'Line__Name']).size().reset_index(name='건수')
counts_groupby.to_excel(f"../excel-output/df_production_컬럼값_[item & Line__Name]_groupby-{len(counts_groupby)}-건.xlsx", index=True)

## Production Head 구분 기준

In [ ]:
counts_groupby = df_production_sample.groupby(['Program', 'Order__Type', 'Channel', '(Contractual)__Ex__Fty__Month']).size().reset_index(name='건수')
counts_groupby.to_excel(f"../excel-output/df_production_컬럼값_[Program & Order__Type & Channel]_groupby-{len(counts_groupby)}-건.xlsx", index=True)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
#
# html = df_production_sample.to_html(index=False)
# Path("df_simple.html").write_text(html, encoding="utf-8")

from pydantic_settings import BaseSettings

import pandas as pd
from pandas_profiling import ProfileReport

profile = ProfileReport(df_production_sample)

profile.to_file(output_file=f"df_production_sample.html") # html


In [ ]:
# !pip uninstall pandas-profiling
!pip install pandas-profiling

In [ ]:
!python3 -m pip install --upgrade pip

In [ ]:
 !pip install pydantic-settings